# Process Gene Expression Dataframe

Processing gene expression dataframe from Xena for downstream analysis

In [12]:
import rnaseq_lib as r
import pandas as pd
import os

- Expected counts downloaded from [Xena](https://xenabrowser.net/datapages/?host=https://toil.xenahubs.net)
- Metadata table processed from [this notebook](https://github.com/jvivian/ipython_notebooks/tree/master/tcga-gtex-metadata)

## Inputs

In [2]:
%%time
df_path = '/mnt/rna-seq-analysis/data/xena/TcgaTargetGtex_gene_expected_count'
met_path = '/mnt/rna-seq-analysis/metadata/tcga_gtex_metadata_intersect.tsv'
df = pd.read_csv(df_path, sep='\t', index_col=0)
met = pd.read_csv(met_path, sep='\t', index_col=0)

CPU times: user 16min 37s, sys: 2min 9s, total: 18min 47s
Wall time: 18min 57s


These files are too big for github, so set directory where all intermediate files will be written

In [3]:
workdir = '/mnt/gene-expression-processing'

## Process Dataframe

### I. Subset TCGA and GTEx samples

In [4]:
tcga_samples = [x for x in df.columns if x.startswith('TCGA')]
gtex_samples = [x for x in df.columns if x.startswith('GTEX')]
print 'TCGA Samples: {}\tGTEx Samples: {}'.format(len(tcga_samples), len(gtex_samples))

TCGA Samples: 10830	GTEx Samples: 7776


In [6]:
df = df[tcga_samples + gtex_samples]
df.shape

(60498, 18606)

### II. Reverse Xena normalization

Xena normalizes data: log2(x + 1). Reverse normalization to get expected counts

In [8]:
%%time
df = df.apply(lambda x: 2**x - 1)

CPU times: user 1min 4s, sys: 23.2 s, total: 1min 28s
Wall time: 1min 28s


save intermediate

In [13]:
r.utils.mkdir_p(workdir)
out = os.path.join(workdir, 'TCGA-GTEx-RSEM-gene-exp-counts.tsv')
df.to_csv(out, sep='\t')

### III. Subset Protein-Coding Genes

In [26]:
genes = []
gtf_path = '/mnt/rna-seq-analysis/metadata/gencode.v23.annotation.gtf'
with open(gtf_path, 'r') as f:
    for line in f:
        if line.startswith('#'):
            continue
        line = r.gtf.parse(line)
        if line['feature'] == 'gene':
            if line['gene_type'] == 'protein_coding':
                genes.append(line['gene_id'])
print '{} protein-coding genes found'.format(len(genes))

19797 protein-coding genes found


Save intermediate

In [29]:
df = df.loc[genes]
out = os.path.join(workdir, 'TCGA-GTEx-RSEM-counts-protein-coding.tsv')
df.to_csv(out, sep='\t')

Save TCGA and GTEx dataframes for normalizing

In [33]:
tcga_path = os.path.join(workdir, 'TCGA-RSEM-counts-protein-coding.tsv')
gtex_path = os.path.join(workdir, 'GTEx-RSEM-counts-protein-coding.tsv')
%time df[tcga_samples].to_csv(tcga_path, sep='\t')
%time df[gtex_samples].to_csv(gtex_path, sep='\t')

CPU times: user 3min 17s, sys: 33.2 s, total: 3min 50s
Wall time: 3min 52s
CPU times: user 2min 13s, sys: 19.5 s, total: 2min 32s
Wall time: 2min 33s


### IV. Run DESeq2 Normalization

In [34]:
%time tcga_norm = r.R.deseq2_normalize(df_path=gtex_path, output_dir=workdir, map_gene_names=False)


Calling: docker run --rm -v /mnt/gene-expression-processing:/data -v /mnt/gene-expression-processing:/df jvivian/deseq2 /data/work_dir/deseq2.R

CPU times: user 124 ms, sys: 1.7 s, total: 1.82 s
Wall time: 25min 27s


'/mnt/gene-expression-processing/GTEx-RSEM-counts-protein-coding.deseq2-normalized.tsv'

In [35]:
%time gtex_norm = r.R.deseq2_normalize(df_path=tcga_path, output_dir=workdir, map_gene_names=False)


Calling: docker run --rm -v /mnt/gene-expression-processing:/data -v /mnt/gene-expression-processing:/df jvivian/deseq2 /data/work_dir/deseq2.R

CPU times: user 256 ms, sys: 1.54 s, total: 1.79 s
Wall time: 47min 36s


'/mnt/gene-expression-processing/TCGA-RSEM-counts-protein-coding.deseq2-normalized.tsv'

### V. Combine and Upload Final Table

In [38]:
df = pd.concat([pd.read_csv(tcga_norm, index_col=0, sep='\t'), 
                pd.read_csv(gtex_norm, index_col=0, sep='\t')], axis=1)
df.shape

(19797, 18606)

In [40]:
df_path = os.path.join(workdir, 'TCGA-GTEx-RSEM-counts-protein-coding.deseq2-normalized.tsv')
df.to_csv(df_path, sep='\t')

In [ ]:
r.web.synapse.upload_file(df_path, login='jtvivian@gmail.com', parent=r.web.synapse.expression)